In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import os
dir_origin = '/content/drive/MyDrive/Smartech/MinaRumas/'
os.chdir(dir_origin)
%pwd

'/content/drive/MyDrive/Smartech/MinaRumas'

In [38]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.147 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.5/112.6 GB disk)


In [39]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import time

In [40]:
def is_point_in_polygon(point, polygon):
    """Verifica si un punto está dentro de un polígono usando el algoritmo de Ray Casting"""
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

In [41]:
def calculate_intersection(box, mask_points):
    """Calcula si hay intersección entre un bounding box y una máscara"""
    # Convertir el bounding box a formato [x1, y1, x2, y2]
    x1, y1, x2, y2 = box

    # Verificar si algún punto de la máscara está dentro del bounding box
    for point in mask_points:
        px, py = point
        if x1 <= px <= x2 and y1 <= py <= y2:
            return True

    # Verificar si alguna esquina del bbox está dentro de la máscara
    corners = [(x1, y1), (x1, y2), (x2, y1), (x2, y2)]
    for corner in corners:
        if is_point_in_polygon(corner, mask_points):
            return True

    return False

In [42]:
def put_text_with_background(img, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.4,
                            color=(255,255,255), thickness=1, bg_color=(0,0,0), bg_alpha=0.6):
    """Coloca texto con fondo semitransparente para mejor legibilidad"""
    # Obtener tamaño del texto
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)

    # Crear imagen para el fondo semitransparente
    x, y = position
    bg_img = img.copy()
    padding = 5  # Padding alrededor del texto

    # Dibujar rectángulo para el fondo
    cv2.rectangle(bg_img, (x-padding, y-text_height-padding),
                 (x+text_width+padding, y+padding), bg_color, -1)

    # Aplicar transparencia
    overlay = cv2.addWeighted(bg_img, bg_alpha, img, 1-bg_alpha, 0)

    # Poner el texto sobre el fondo semitransparente
    cv2.putText(overlay, text, (x, y), font, font_scale, color, thickness)

    return overlay


In [43]:
video_path = 'video_cam1.mp4'  # Ruta real de tu video
output_path = 'processed_video_demo___v3.mp4'
start_time_sec = 10
end_time_sec = 20

model_det = YOLO('/content/drive/MyDrive/Smartech/MinaRumas/model_detection.pt')
model_seg = YOLO('/content/drive/MyDrive/Smartech/MinaRumas/model_segmentation.pt')

In [44]:
# Definir la zona de detección
detection_zone = np.array([[20, 691], [18, 779], [414, 881], [709, 759], [1675, 1060], [1902, 890], [1902, 667], [704, 416]], np.int32)
detection_zone = detection_zone // 2
# Abrir el video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(width)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(height)
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
start_frame = int(start_time_sec * fps)
print(f"Frame inicial: {start_frame}")
end_frame = int(end_time_sec * fps)
print(f"Frame final: {end_frame}")

# Inicializar variables
frame_count = 0
initial_masks = {}  # Almacenar las máscaras iniciales de las rumas
initial_areas = {}  # Almacenar las áreas iniciales de las rumas
ruma_classes = {0: 'ruma'}
det_classes = {0: 'person', 1: 'vehicle'}

# Colores para las visualizaciones
RUMA_COLOR = (0, 255, 0)  # Verde para rumas
PERSON_COLOR = (255, 255, 0)  # Celeste (BGR)
VEHICLE_COLOR = (0, 0, 255)  # Rojo para vehículos
TEXT_COLOR_WHITE = (255, 255, 255)
TEXT_COLOR_GREEN = (0, 255, 0)
TEXT_COLOR_RED = (0, 0, 255)

848
478
Frame inicial: 300
Frame final: 600


In [45]:
with torch.no_grad():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count > end_frame:
            break

        if frame_count >= start_frame:
            # Hacer una copia del frame para dibujar
            frame_ext = frame.copy()

            # Inicializar flags para los mensajes de estado
            object_in_zone = False
            object_interacting = False
            ruma_variation = False

            # Inicializar un diccionario para rastrear las rumas que interactúan con objetos
            rumas_interacting = {}

            # Ejecutar detección de personas y vehículos primero para identificar interacciones
            result_det = model_det.track(frame, conf=0.5, persist=True, verbose=False)

            # Procesar resultados de detección de personas y vehículos
            if result_det and len(result_det) > 0:
                boxes = result_det[0].boxes

                if boxes is not None and len(boxes) > 0:
                    for box in boxes:
                        # Obtener coordenadas y clase
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        cls = int(box.cls[0])
                        conf = float(box.conf[0])

                        if conf > 0.5:
                            # Definir color según la clase
                            color = PERSON_COLOR if cls == 0 else VEHICLE_COLOR
                            label = 'persona' if cls == 0 else 'maquinaria'

                            # Dibujar bounding box
                            cv2.rectangle(frame_ext, (x1, y1), (x2, y2), color, 2)

                            # Añadir etiqueta con fondo semitransparente
                            frame_ext = put_text_with_background(
                                frame_ext,
                                label,
                                (x1, y1 - 5),
                                color=TEXT_COLOR_WHITE,
                                font_scale=0.6,
                            )

                            # Verificar si el objeto está en la zona de detección
                            center_x = (x1 + x2) // 2
                            center_y = (y1 + y2) // 2

                            # Verificar si el centro está dentro de la zona de detección
                            if is_point_in_polygon((center_x, center_y), detection_zone):
                                object_in_zone = True

                            # Verificar interacción con rumas
                            for ruma_id, mask in initial_masks.items():
                                if calculate_intersection([x1, y1, x2, y2], mask):
                                    object_interacting = True
                                    # Marcar esta ruma como interactuando con un objeto
                                    rumas_interacting[ruma_id] = True

            # Ejecutar segmentación de rumas
            result_seg = model_seg(frame, conf=0.5, verbose=False)

            # Procesar resultados de segmentación de rumas
            if result_seg and len(result_seg) > 0:
                for r in result_seg:
                    if r.masks is not None:
                        masks = r.masks.xy

                        # Procesar cada máscara (ruma)
                        for i, mask in enumerate(masks):
                            # Generar un ID para la ruma basado en su centroide
                            centroid_x = int(np.mean([p[0] for p in mask]))
                            centroid_y = int(np.mean([p[1] for p in mask]))
                            ruma_id = f"ruma_{centroid_x}_{centroid_y}"

                            if is_point_in_polygon((centroid_x, centroid_y), detection_zone) == False:
                                continue

                            # Si es el primer frame, guardar la máscara y área inicial
                            if frame_count == start_frame:
                                initial_masks[ruma_id] = mask
                                initial_areas[ruma_id] = cv2.contourArea(mask.astype(np.int32))

                            # Calcular el área actual
                            current_area = cv2.contourArea(mask.astype(np.int32))

                            # Encontrar la máscara inicial más cercana a esta
                            best_match = None
                            min_distance = float('inf')

                            for rid, imask in initial_masks.items():
                                # Calcular distancia entre centroides
                                i_centroid_x = int(np.mean([p[0] for p in imask]))
                                i_centroid_y = int(np.mean([p[1] for p in imask]))

                                dist = ((centroid_x - i_centroid_x)**2 + (centroid_y - i_centroid_y)**2)**0.5

                                if dist < min_distance:
                                    min_distance = dist
                                    best_match = rid

                            # Si encontramos una coincidencia y está lo suficientemente cerca
                            if best_match and min_distance < 100:  # Umbral de distancia
                                ruma_id = best_match
                                initial_area = initial_areas[ruma_id]
                                initial_mask = initial_masks[ruma_id]

                                # Calcular el porcentaje del área
                                area_percentage = (current_area / initial_area) * 100

                                # Rellenar la máscara con un verde semitransparente
                                overlay = frame_ext.copy()
                                cv2.fillPoly(overlay, [mask.astype(np.int32)], RUMA_COLOR)
                                frame_ext = cv2.addWeighted(overlay, 0.3, frame_ext, 0.7, 0)

                                # Determinar si debemos mostrar la variación o 100%
                                # Solo mostramos variación si la ruma está interactuando con un objeto y tiene variación < 95%
                                is_interacting = ruma_id in rumas_interacting
                                has_variation = area_percentage < 95

                                if is_interacting and has_variation:
                                    percentage_text = f"{area_percentage:.1f}%"
                                    ruma_variation = True
                                else:
                                    percentage_text = "100%"

                                # Centroide de la mascara inicial para el texto
                                first_centroid_x = int(np.mean([p[0] for p in initial_mask]))
                                first_centroid_y = int(np.mean([p[1] for p in initial_mask]))

                                # Poner texto con fondo semitransparente
                                frame_ext = put_text_with_background(
                                    frame_ext,
                                    percentage_text,
                                    (first_centroid_x, first_centroid_y),
                                    font_scale=0.6,
                                    color=TEXT_COLOR_WHITE
                                )

            # Dibujar la zona de detección con líneas punteadas
            pts = detection_zone.reshape((-1, 1, 2))
            cv2.polylines(frame_ext, [pts], True, (0, 255, 255), 2, lineType=cv2.LINE_AA)

            # Añadir textos informativos en la esquina superior derecha
            text_y_start = 50

            # Texto 1: Detección en zona
            zone_text = "Movimiento en la zona" if object_in_zone else "Zona despejada"
            zone_color = TEXT_COLOR_RED if object_in_zone else TEXT_COLOR_GREEN
            frame_ext = put_text_with_background(
                frame_ext,
                zone_text,
                (width - 650, text_y_start),
                color=zone_color,
                font_scale=1.5
            )

            # Texto 2: Interacción con rumas
            interact_text = "Interaccion con las rumas" if object_interacting else "Sin interacciones"
            interact_color = TEXT_COLOR_RED if object_interacting else TEXT_COLOR_GREEN
            frame_ext = put_text_with_background(
                frame_ext,
                interact_text,
                (width - 650, text_y_start + 60),
                color=interact_color,
                font_scale=1.5
            )

            # Texto 3: Variación de rumas (solo muestra variación real cuando hay interacción)
            variation_text = "Variacion en las rumas" if ruma_variation else "Rumas en reposo"
            variation_color = TEXT_COLOR_RED if ruma_variation else TEXT_COLOR_GREEN
            frame_ext = put_text_with_background(
                frame_ext,
                variation_text,
                (width - 650, text_y_start + 120),
                color=variation_color,
                font_scale=1.5
            )

            # Escribir el frame procesado al video de salida
            out.write(frame_ext)
        frame_count += 1

        # Mostrar progreso cada 50 frames
        if frame_count % 50 == 0:
            print(f"Procesados {frame_count} frames")

# Liberar recursos
cap.release()
out.release()
# cv2.destroyAllWindows()
print(f"Procesamiento completado. Video guardado en {output_path}")

Procesados 50 frames
Procesados 100 frames
Procesados 150 frames
Procesados 200 frames
Procesados 250 frames
Procesados 300 frames
Procesados 350 frames
Procesados 400 frames
Procesados 450 frames
Procesados 500 frames
Procesados 550 frames
Procesados 600 frames
Procesamiento completado. Video guardado en processed_video_demo___v3.mp4
